# Time-Series Anomaly Detection using Implicit Neural Representations

In [ ]:
from tqdm import tqdm_notebook
from itertools import product
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch._C import device
import time
device = torch.device(f"cuda:{0}" if torch.cuda.is_available() else "cpu")


In [ ]:
import numpy as np


def calc_point2point(predict, actual):
    """
    calculate f1 score by predict and actual.

    Args:
        predict (np.ndarray): the predict label
        actual (np.ndarray): np.ndarray
    """
    TP = np.sum(predict * actual)
    TN = np.sum((1 - predict) * (1 - actual))
    FP = np.sum(predict * (1 - actual))
    FN = np.sum((1 - predict) * actual)
    precision = TP / (TP + FP + 0.00001)
    recall = TP / (TP + FN + 0.00001)
    f1 = 2 * precision * recall / (precision + recall + 0.00001)
    return f1, precision, recall, TP, TN, FP, FN


def adjust_predicts(score, label, threshold=None, pred=None, calc_latency=False):
    """
    Calculate adjusted predict labels using given `score`, `threshold` (or given `pred`) and `label`.

    Args:
        score (np.ndarray): The anomaly score
        label (np.ndarray): The ground-truth label
        threshold (float): The threshold of anomaly score.
            A point is labeled as "anomaly" if its score is higher than the threshold.
        pred (np.ndarray or None): if not None, adjust `pred` and ignore `score` and `threshold`,
        calc_latency (bool):

    Returns:
        np.ndarray: predict labels
    """
    if len(score) != len(label):
        raise ValueError("score and label must have the same length")
    score = np.asarray(score)
    label = np.asarray(label)
    latency = 0
    if pred is None:
        predict = score > threshold
    else:
        predict = pred
    actual = label > 0.1
    anomaly_state = False
    anomaly_count = 0
    for i in range(len(score)):
        if actual[i] and predict[i] and not anomaly_state:
            anomaly_state = True
            anomaly_count += 1
            for j in range(i, 0, -1):
                if not actual[j]:
                    break
                else:
                    if not predict[j]:
                        predict[j] = True
                        latency += 1
        elif not actual[i]:
            anomaly_state = False
        if anomaly_state:
            predict[i] = True
    if calc_latency:
        return predict, latency / (anomaly_count + 1e-4)
    else:
        return predict


def calc_seq(score, label, threshold, calc_latency=False):
    """
    Calculate f1 score for a score sequence
    """
    if calc_latency:
        predict, latency = adjust_predicts(
            score, label, threshold, calc_latency=calc_latency
        )
        t = list(calc_point2point(predict, label))
        t.append(latency)
        return t
    else:
        predict = adjust_predicts(score, label, threshold, calc_latency=calc_latency)
        return calc_point2point(predict, label)


def bf_search(score, label, verbose=False):
    """
    Find the best-f1 score by searching best `threshold` in [`start`, `end`).

    Returns:
        list: list for results
        float: the `threshold` for best-f1
    """
    candidate = sorted(set(score))
    m = (-1.0, -1.0, -1.0)
    m_t = 0.0
    for threshold in candidate:
        target = calc_seq(score, label, threshold, calc_latency=True)
        # If f1 score increases
        if target[0] > m[0]:
            m_t = threshold
            m = target
    if verbose:
        print(m, m_t)
    return m, m_t

In [ ]:
import pickle
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

prefix = "processed"


def save_z(z, filename="z"):
    """
    save the sampled z in a txt file
    """
    for i in range(0, z.shape[1], 20):
        with open(filename + "_" + str(i) + ".txt", "w") as file:
            for j in range(0, z.shape[0]):
                for k in range(0, z.shape[2]):
                    file.write("%f " % (z[j][i][k]))
                file.write("\n")
    i = z.shape[1] - 1
    with open(filename + "_" + str(i) + ".txt", "w") as file:
        for j in range(0, z.shape[0]):
            for k in range(0, z.shape[2]):
                file.write("%f " % (z[j][i][k]))
            file.write("\n")


def get_data_dim(dataset):
    if dataset == "SMAP":
        return 25
    elif dataset == "MSL":
        return 55
    elif str(dataset).startswith("machine"):
        return 38
    else:
        raise ValueError("unknown dataset " + str(dataset))


def get_data(
    prefix,
    dataset,
    max_train_size=None,
    max_test_size=None,
    print_log=True,
    do_preprocess=False,
    train_start=0,
    test_start=0,
):
    """
    get data from pkl files

    return shape: (([train_size, x_dim], [train_size] or None), ([test_size, x_dim], [test_size]))
    """
    if max_train_size is None:
        train_end = None
    else:
        train_end = train_start + max_train_size
    if max_test_size is None:
        test_end = None
    else:
        test_end = test_start + max_test_size
    # print("load data of:", dataset)
    # print("train: ", train_start, train_end)
    # print("test: ", test_start, test_end)
    x_dim = get_data_dim(dataset)
    # print(dataset)
    f = open(os.path.join(prefix, dataset + "_train.pkl"), "rb")
    train_data = pickle.load(f).reshape((-1, x_dim))[train_start:train_end, :]
    f.close()
    try:
        f = open(os.path.join(prefix, dataset + "_test.pkl"), "rb")
        test_data = pickle.load(f).reshape((-1, x_dim))[test_start:test_end, :]
        f.close()
    except (KeyError, FileNotFoundError):
        test_data = None
    try:
        f = open(os.path.join(prefix, dataset + "_test_label.pkl"), "rb")
        test_label = pickle.load(f).reshape((-1))[test_start:test_end]
        f.close()
    except (KeyError, FileNotFoundError):
        test_label = None
    if do_preprocess:
        train_data = preprocess(train_data)
        test_data = preprocess(test_data)

    # print("train set shape: ", train_data.shape)
    # print("test set shape: ", test_data.shape)
    # print("test set label shape: ", test_label.shape)
    return (train_data, None), (test_data, test_label)


def preprocess(df):
    """returns normalized and standardized data."""

    df = np.asarray(df, dtype=np.float32)

    if len(df.shape) == 1:
        raise ValueError("Data must be a 2-D array")

    if np.any(sum(np.isnan(df)) != 0):
        print("Data contains null values. Will be replaced with 0")
        df = np.nan_to_num()

    # normalize data
    df = MinMaxScaler().fit_transform(df)
    # print("Data normalized")

    return df


class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path="checkpoint.pt"):

        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model=None):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        #             self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            #             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            #             self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model=None):
        if self.verbose:
            print(
                f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ..."
            )
        #         torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from collections import OrderedDict
import utils
import pandas as pd

# SIREN code reference link: https://github.com/vsitzmann/siren


class SineLayer(torch.nn.Module):

    # If is_first=True, omega_0 is a frequency factor which simply multiplies the activations before the
    # nonlinearity. Different signals may require different omega_0 in the first layer - this is a
    # hyperparameter.

    # If is_first=False, then the weights will be divided by omega_0 so as to keep the magnitude of
    # activations constant, but boost gradients to the weight matrix (see supplement Sec. 1.5)

    def __init__(
        self, in_features, out_features, bias=True, is_first=False, omega_0=30
    ):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first

        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)

        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 1 / self.in_features)
            else:
                self.linear.weight.uniform_(
                    -np.sqrt(6 / self.in_features) / self.omega_0,
                    np.sqrt(6 / self.in_features) / self.omega_0,
                )

    def forward(self, input):
        return torch.sin(self.omega_0 * self.linear(input))

    def forward_with_intermediate(self, input):
        # For visualization of activation distributions
        intermediate = self.omega_0 * self.linear(input)
        return torch.sin(intermediate), intermediate


class Siren(torch.nn.Module):
    def __init__(
        self,
        in_features,
        hidden_features,
        hidden_layers,
        out_features,
        outermost_linear=True,
        first_omega_0=3000,
        hidden_omega_0=30.0,
    ):
        super().__init__()

        self.net = []
        self.net.append(
            SineLayer(
                in_features, hidden_features, is_first=True, omega_0=first_omega_0
            )
        )

        for i in range(hidden_layers):
            self.net.append(
                SineLayer(
                    hidden_features,
                    hidden_features,
                    is_first=False,
                    omega_0=hidden_omega_0,
                )
            )

        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)

            with torch.no_grad():
                final_linear.weight.uniform_(
                    -np.sqrt(6 / hidden_features) / hidden_omega_0,
                    np.sqrt(6 / hidden_features) / hidden_omega_0,
                )

            self.net.append(final_linear)
        else:
            self.net.append(
                SineLayer(
                    hidden_features,
                    out_features,
                    is_first=False,
                    omega_0=hidden_omega_0,
                )
            )

        self.net = nn.Sequential(*self.net)

    def forward(self, coords):
        coords = (
            coords.clone().detach().requires_grad_(True)
        )  # allows to take derivative w.r.t. input
        output = self.net(coords)
        return output, coords

    def forward_with_activations(self, coords, retain_grad=False):
        """Returns not only model output, but also intermediate activations.
        Only used for visualizing activations later!"""
        activations = OrderedDict()

        activation_count = 0
        x = coords.clone().detach().requires_grad_(True)
        activations["input"] = x
        for i, layer in enumerate(self.net):
            if isinstance(layer, SineLayer):
                x, intermed = layer.forward_with_intermediate(x)

                if retain_grad:
                    x.retain_grad()
                    intermed.retain_grad()

                activations[
                    "_".join((str(layer.__class__), "%d" % activation_count))
                ] = intermed
                activation_count += 1
            else:
                x = layer(x)

                if retain_grad:
                    x.retain_grad()

            activations["_".join((str(layer.__class__), "%d" % activation_count))] = x
            activation_count += 1

        return activations


# Source code for our newly designed temporal encoding technique


class Timedata(torch.utils.data.Dataset):
    def __init__(self, data, encoded_input, train_scale=None):
        self.data = data
        self.scale = train_scale
        self.timepoints = encoded_input
        self.data_ready = self.data
        if self.scale is None:
            self.scale = np.max(np.abs(self.data_ready))
        self.data_ready = self.data_ready / self.scale
        self.data_ready = torch.Tensor(self.data_ready).view(
            -1, self.data.shape[1]
        )  # dimension change

    def get_num_samples(self):
        return self.timepoints.shape[0]

    def __len__(self):
        return self.timepoints.shape[0]

    def __getitem__(self, idx):
        return self.timepoints[idx, :], self.data_ready[idx, :]


def timestamp_maker(total_length, start=None, unit="1min"):
    if start == None:
        start = "1/1/2021"
    timestamp = pd.date_range(start, periods=total_length, freq=unit)
    return timestamp


def mapping(x, a, b):
    # x = array, list
    # a,b : list
    new_x = []
    for i in x:
        j = b[a.index(i)]
        new_x.append(j)
    return torch.Tensor(new_x)


def temporal_encoding(timestamp, overyear_prediction=None):

    df = pd.DataFrame(index=timestamp)

    df["y"] = timestamp.year
    df["m"] = timestamp.month
    df["d"] = timestamp.day
    # df['w'] = timestamp.weekday
    df["h"] = timestamp.hour
    df["min"] = timestamp.minute
    df["s"] = timestamp.second
    unit_needed = ["y", "m", "d", "h", "min", "s"]
    print(df)
    # dic = df.nunique(dropna=True)
    # unit_needed = [i for i in dic.keys() if dic[i] > 1]

    # discrete values for basic time scale with 5 kinds (year has no discrete limit.)
    m_num = 12
    d_num = 31
    h_num = 24
    min_num = 60
    s_num = 60

    if overyear_prediction is None:
        overyear_prediction = 1

    actual_values = {}
    normalized_value = {}
    for i in unit_needed:
        if i == "y":
            value = list(
                range(
                    min(df[i].values), max(df[i].values) + overyear_prediction
                )  # first_year ~ first_year + overyear_prediction
            )
            stamp = torch.linspace(
                -1, 1, steps=len(df["y"].value_counts().keys()) + overyear_prediction
            )
        elif i == "m":
            value = list(range(1, 1 + m_num))  # 1~12
            stamp = torch.linspace(-1, 1, steps=m_num)
        elif i == "d":
            value = list(range(1, 1 + d_num))  # 1~31
            stamp = torch.linspace(-1, 1, steps=d_num)
        elif i == "h":
            value = list(range(h_num))  # 0~23
            stamp = torch.linspace(-1, 1, steps=h_num)
        elif i == "min":
            value = list(range(min_num))  # 0~59
            stamp = torch.linspace(-1, 1, steps=min_num)
        elif i == "s":
            value = list(range(s_num))  # 0~59
            stamp = torch.linspace(-1, 1, steps=s_num)
        normalized_value[i] = stamp
        actual_values[i] = value
    encoded_time_input = []
    for i in unit_needed:
        encoded_time_input.append(
            mapping(df[i].values, actual_values[i], normalized_value[i])
        )
    encoded_time_input = torch.stack(encoded_time_input, dim=1)
    return encoded_time_input


## Dataset

In [3]:
filepath = 'https://raw.githubusercontent.com/selva86/datasets/master/Raotbl6.csv'
macro_data = pd.read_csv(filepath, parse_dates=['date'], index_col='date')
print(macro_data.shape)  # (123, 8)
macro_data.head()

(123, 8)


,rgnp,pgnp,ulc,gdfco,gdf,gdfim,gdfcf,gdfce
date,,,,,,,,
1959-01-01,1606.4,1608.3,47.5,36.9,37.4,26.9,32.3,23.1
1959-04-01,1637.0,1622.2,47.5,37.4,37.5,27.0,32.2,23.4
1959-07-01,1629.5,1636.2,48.7,37.6,37.6,27.1,32.4,23.4
1959-10-01,1643.4,1650.3,48.8,37.7,37.8,27.1,32.5,23.8
1960-01-01,1671.6,1664.6,49.1,37.8,37.8,27.2,32.4,23.8


In [4]:
date=pd.to_datetime(macro_data.index)

In [5]:
train,test = train_test_split(macro_data, test_size=0.2, random_state=42)

In [6]:
train.shape

(98, 8)

In [7]:
test.shape

(25, 8)

In [8]:
date_train=date[:98]
date_test=date[98:]

In [9]:
date_train.shape

(98,)

In [10]:
date_test.shape

(25,)

## Encode the timstamps

In [ ]:
train_encoded_input = temporal_encoding(date_train)
test_encoded_input= temporal_encoding(date_test)

               y   m  d  h  min  s
date                              
1959-01-01  1959   1  1  0    0  0
1959-04-01  1959   4  1  0    0  0
1959-07-01  1959   7  1  0    0  0
1959-10-01  1959  10  1  0    0  0
1960-01-01  1960   1  1  0    0  0
...          ...  .. .. ..  ... ..
1982-04-01  1982   4  1  0    0  0
1982-07-01  1982   7  1  0    0  0
1982-10-01  1982  10  1  0    0  0
1983-01-01  1983   1  1  0    0  0
1983-04-01  1983   4  1  0    0  0

[98 rows x 6 columns]
               y   m  d  h  min  s
date                              
1983-07-01  1983   7  1  0    0  0
1983-10-01  1983  10  1  0    0  0
1984-01-01  1984   1  1  0    0  0
1984-04-01  1984   4  1  0    0  0
1984-07-01  1984   7  1  0    0  0
1984-10-01  1984  10  1  0    0  0
1985-01-01  1985   1  1  0    0  0
1985-04-01  1985   4  1  0    0  0
1985-07-01  1985   7  1  0    0  0
1985-10-01  1985  10  1  0    0  0
1986-01-01  1986   1  1  0    0  0
1986-04-01  1986   4  1  0    0  0
1986-07-01  1986   7  1  0    0 

In [15]:
train=train.values
test=test.values

## Train the model on the train dataset

In [ ]:
hidden_dim=256
batch_size=32 # 32)
epochs=1 # For simplicity, we set it as 1, however originally we set it as 10000
earlystopping_patience=30
first_omega_0=3000

# Model initialization

model = Siren(
    in_features=train_encoded_input.shape[1],
    out_features=train.shape[1],
    hidden_features=hidden_dim,
    hidden_layers=3,
    first_omega_0=first_omega_0,
    outermost_linear=True,
)
model.to(device)

optim = torch.optim.Adam(lr=1e-4, params=model.parameters())

In [ ]:
data_train = Timedata(train, train_encoded_input)
train_dataloader = DataLoader(
    data_train,
    shuffle=True,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=0,
)

early_stopping = EarlyStopping(
    patience=earlystopping_patience, verbose=False
)

epoch_time = []
for step in range(epochs):
    epoch_start = time.time()
    model_loss = 0
    for batch_model_input, batch_ground_truth in train_dataloader:
        batch_model_input = batch_model_input.to(device)
        batch_ground_truth = batch_ground_truth.to(device)

        batch_model_output, _ = model(batch_model_input)
        loss = F.mse_loss(batch_model_output, batch_ground_truth)
        optim.zero_grad()
        loss.backward()
        optim.step()
        model_loss += loss.item()
        batch_model_input = batch_model_input.detach().cpu()
        batch_ground_truth = batch_ground_truth.detach().cpu()
    epoch_time.append(time.time() - epoch_start)
    early_stopping(model_loss)
    if early_stopping.early_stop:
        break

print("average training time per epoch: ", np.mean(epoch_time))

average training time per epoch:  0.14271235466003418


## Retrain the model on the test set

In [ ]:
data_test = Timedata(test, test_encoded_input)
test_dataloader = DataLoader(
    data_test,
    shuffle=True,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=0,
)

early_stopping = utils.EarlyStopping(
    patience=earlystopping_patience, verbose=False
)

print("re-training start")
for step in range(epochs):
    epoch_start = time.time()
    model_loss = 0
    for batch_model_input, batch_ground_truth in train_dataloader:
        batch_model_input = batch_model_input.to(device)
        batch_ground_truth = batch_ground_truth.to(device)
        batch_model_output, _ = model(batch_model_input)
        loss = F.mse_loss(batch_model_output, batch_ground_truth)
        optim.zero_grad()
        loss.backward()
        optim.step()
        model_loss += loss.item()
        batch_model_input = batch_model_input.detach().cpu()
        batch_ground_truth = batch_ground_truth.detach().cpu()
    early_stopping(model_loss)
    if early_stopping.early_stop:
        break
print("re-training end")

re-training start
re-training end


In [20]:
total_input = data_test.timepoints
model = model.cpu()
total_ground_truth = data_test.data_ready
total_model_output, _ = model(total_input)

anomaly_score = np.mean(
    np.abs(
        np.squeeze(
            total_ground_truth.numpy()
            - total_model_output.detach().cpu().numpy()
        )
    ),
    axis=1,
)


### Threshold

to find the nest threshold we us the function bf_search to search for the best F1 score corresponding to the best threshold.but the data need to be labeled and I did,'t find any labeled dataset

In [ ]:
accuracy, threshold = bf_search(anomaly_score, y_test, verbose = False)
print("Precision: {}, Recall {}, F1-score: {}".format(accuracy[1], accuracy[2], accuracy[0]))

(25,)